In [1]:
from __future__ import division
from nltk.book import *
import nltk
import numpy
import matplotlib
import nltk, re, pprint
from urllib import urlopen
import sys
#reload(sys)                       <----- activate this for reading 
#sys.setdefaultencoding('utf8')

from random import randint
from bs4 import BeautifulSoup

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [2]:
len(nltk.corpus.brown.words()) / len(nltk.corpus.brown.sents())

20.250994070456922

In [3]:
sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')

In [4]:
text = nltk.corpus.gutenberg.raw('chesterton-thursday.txt')

In [5]:
sents = sent_tokenizer.tokenize(text)

In [6]:
pprint.pprint(sents[171:181])

[u'In the wild events which were to follow this girl had no\npart at all; he never saw her again until all his tale was over.',
 u'And yet, in some indescribable way, she kept recurring like a\nmotive in music through all his mad adventures afterwards, and the\nglory of her strange hair ran like a red thread through those dark\nand ill-drawn tapestries of the night.',
 u'For what followed was so\nimprobable, that it might well have been a dream.',
 u'When Syme went out into the starlit street, he found it for the\nmoment empty.',
 u'Then he realised (in some odd way) that the silence\nwas rather a living silence than a dead one.',
 u'Directly outside the\ndoor stood a street lamp, whose gleam gilded the leaves of the tree\nthat bent out over the fence behind him.',
 u'About a foot from the\nlamp-post stood a figure almost as rigid and motionless as the\nlamp-post itself.',
 u'The tall hat and long frock coat were black; the\nface, in an abrupt shadow, was almost as dark.',
 u'Only a fr

### Word Segmentations

In [7]:
def segment(text, segs):
    words = []
    last = 0
    for i in range(len(segs)):
        if segs[i] == '1':
            words.append(text[last:i+1])
            last = i+1
    words.append(text[last:])
    return words

In [8]:
text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
seg2 = "0100100100100001001001000010100100010010000100010010000"

len(text)

56

In [9]:
print segment(text, seg1)
print segment(text, seg2)

['doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy']
['do', 'you', 'see', 'the', 'kitty', 'see', 'the', 'doggy', 'do', 'you', 'like', 'the', 'kitty', 'like', 'the', 'doggy']


In [10]:
def evaluate(text, segs):
    words = segment(text, segs)
    text_size = len(words)
    lexicon_size = len(' '.join(list(set(words))))
    print text_size
    print words
    print set(words)
    print list(set(words))
    print ' '.join(list(set(words)))
    print lexicon_size
    return text_size + lexicon_size

In [11]:
evaluate(text,seg1)

4
['doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy']
set(['doyoulikethekitty', 'doyouseethekitty', 'likethedoggy', 'seethedoggy'])
['doyoulikethekitty', 'doyouseethekitty', 'likethedoggy', 'seethedoggy']
doyoulikethekitty doyouseethekitty likethedoggy seethedoggy
59


63

In [12]:
text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"
seg2 = "0100100100100001001001000010100100010010000100010010000"
seg3 = "0000100100000011001000000110000100010000001100010000001"

In [13]:
segment(text, seg3)

['doyou',
 'see',
 'thekitt',
 'y',
 'see',
 'thedogg',
 'y',
 'doyou',
 'like',
 'thekitt',
 'y',
 'like',
 'thedogg',
 'y']

In [14]:
print evaluate(text, seg3)
print evaluate(text, seg2)
print evaluate(text, seg1)

14
['doyou', 'see', 'thekitt', 'y', 'see', 'thedogg', 'y', 'doyou', 'like', 'thekitt', 'y', 'like', 'thedogg', 'y']
set(['thedogg', 'like', 'doyou', 'see', 'y', 'thekitt'])
['thedogg', 'like', 'doyou', 'see', 'y', 'thekitt']
thedogg like doyou see y thekitt
32
46
16
['do', 'you', 'see', 'the', 'kitty', 'see', 'the', 'doggy', 'do', 'you', 'like', 'the', 'kitty', 'like', 'the', 'doggy']
set(['do', 'like', 'kitty', 'see', 'you', 'doggy', 'the'])
['do', 'like', 'kitty', 'see', 'you', 'doggy', 'the']
do like kitty see you doggy the
31
47
4
['doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy']
set(['doyoulikethekitty', 'doyouseethekitty', 'likethedoggy', 'seethedoggy'])
['doyoulikethekitty', 'doyouseethekitty', 'likethedoggy', 'seethedoggy']
doyoulikethekitty doyouseethekitty likethedoggy seethedoggy
59
63


In [15]:
def flip(segs, pos):
    return segs[:pos] + str(1-int(segs[pos])) + segs[pos+1:]
def flip_n(segs, n):
    for i in range(n):
        segs = flip(segs, randint(0,len(segs)-1))
    return segs
def anneal(text, segs, iterations, cooling_rate):
    temperature = float(len(segs))
    while temperature > 0.5:
        best_segs, best = segs, evaluate(text, segs)
        for i in range(iterations):
            guess = flip_n(segs, int(round(temperature)))
            score = evaluate(text, guess)
            if score < best:
                best, best_segs = score, guess
        score, segs = best, best_segs
        temperature = temperature / cooling_rate
        print evaluate(text, segs), segment(text, segs)
    print
    return segs

In [16]:
text = "doyouseethekittyseethedoggydoyoulikethekittylikethedoggy"
seg1 = "0000000000000001000000000010000000000000000100000000000"

In [17]:
seg1 = "0000000000000001000000000010000000000000000100000000000"

4
['doyouseethekitty', 'seethedoggy', 'doyoulikethekitty', 'likethedoggy']
set(['doyoulikethekitty', 'doyouseethekitty', 'likethedoggy', 'seethedoggy'])
['doyoulikethekitty', 'doyouseethekitty', 'likethedoggy', 'seethedoggy']
doyoulikethekitty doyouseethekitty likethedoggy seethedoggy
59
29
['d', 'oyou', 's', 'ee', 't', 'he', 'k', 'i', 't', 't', 'ys', 'ee', 't', 'h', 'e', 'do', 'gg', 'ydoyo', 'u', 'like', 'thek', 'i', 't', 'ty', 'liket', 'h', 'ed', 'ogg', 'y']
set(['ee', 'ed', 'gg', 'liket', 'ydoyo', 'ogg', 'ty', 'oyou', 'he', 'do', 'thek', 'ys', 'like', 'e', 'd', 'i', 'h', 'k', 's', 'u', 't', 'y'])
['ee', 'ed', 'gg', 'liket', 'ydoyo', 'ogg', 'ty', 'oyou', 'he', 'do', 'thek', 'ys', 'like', 'e', 'd', 'i', 'h', 'k', 's', 'u', 't', 'y']
ee ed gg liket ydoyo ogg ty oyou he do thek ys like e d i h k s u t y
69
23
['doyousee', 'th', 'ek', 'it', 'tys', 'e', 'e', 'the', 'dog', 'g', 'yd', 'oyoul', 'i', 'keth', 'e', 'k', 'itty', 'liket', 'hed', 'o', 'g', 'g', 'y']
set(['doyousee', 'e', 'g', 'ek'

23
['d', 'o', 'yous', 'e', 'ethekitty', 'se', 'ethe', 'd', 'oggy', 'd', 'o', 'y', 'o', 'u', 'li', 'k', 'e', 't', 'hekit', 't', 'yliketh', 'edo', 'ggy']
set(['e', 'd', 'hekit', 'ethe', 'ggy', 'edo', 'ethekitty', 'li', 'u', 'o', 'oggy', 'y', 'yliketh', 'yous', 'k', 'se', 't'])
['e', 'd', 'hekit', 'ethe', 'ggy', 'edo', 'ethekitty', 'li', 'u', 'o', 'oggy', 'y', 'yliketh', 'yous', 'k', 'se', 't']
e d hekit ethe ggy edo ethekitty li u o oggy y yliketh yous k se t
66
29
['doyous', 'ee', 't', 'h', 'e', 'k', 'i', 'tt', 'ys', 'ee', 'th', 'edog', 'gy', 'do', 'y', 'o', 'ul', 'i', 'ke', 'th', 'e', 'k', 'i', 'tty', 'li', 'k', 'et', 'hedo', 'ggy']
set(['tty', 'ee', 'gy', 'et', 'tt', 'li', 'hedo', 'th', 'do', 'edog', 'ggy', 'ys', 'e', 'ke', 'i', 'h', 'k', 'doyous', 'o', 'ul', 't', 'y'])
['tty', 'ee', 'gy', 'et', 'tt', 'li', 'hedo', 'th', 'do', 'edog', 'ggy', 'ys', 'e', 'ke', 'i', 'h', 'k', 'doyous', 'o', 'ul', 't', 'y']
tty ee gy et tt li hedo th do edog ggy ys e ke i h k doyous o ul t y
68
25
['doy',

set(['do', 'he', 'thedogg', 'e', 'yo', 'g', 'ki', 'gydo', 'doyous', 'li', 'tyse', 'th', 'y', 'et', 'the', 'ulik', 'tt', 'k', 't'])
['do', 'he', 'thedogg', 'e', 'yo', 'g', 'ki', 'gydo', 'doyous', 'li', 'tyse', 'th', 'y', 'et', 'the', 'ulik', 'tt', 'k', 't']
do he thedogg e yo g ki gydo doyous li tyse th y et the ulik tt k t
67
29
['d', 'o', 'y', 'o', 'use', 'e', 'th', 'ek', 'i', 't', 't', 'y', 's', 'e', 'ethed', 'og', 'gyd', 'oy', 'o', 'uli', 'k', 'e', 'the', 'ki', 'ttyli', 'keth', 'edo', 'g', 'gy']
set(['ek', 'ethed', 'edo', 'gy', 'ttyli', 'use', 'th', 'gyd', 'keth', 'e', 'd', 'g', 'i', 'k', 'o', 'og', 's', 'uli', 't', 'oy', 'y', 'the', 'ki'])
['ek', 'ethed', 'edo', 'gy', 'ttyli', 'use', 'th', 'gyd', 'keth', 'e', 'd', 'g', 'i', 'k', 'o', 'og', 's', 'uli', 't', 'oy', 'y', 'the', 'ki']
ek ethed edo gy ttyli use th gyd keth e d g i k o og s uli t oy y the ki
72
19
['do', 'you', 'se', 'etheki', 'ttys', 'eethed', 'oggydoy', 'o', 'ul', 'ik', 'et', 'h', 'ek', 'i', 't', 'tylike', 't', 'he', 'd

['o', 'ethekitt', 'd', 'ee', 's', 'k', 'kethed', 'doyou', 'gy', 'lik', 'gg', 'og', 'see', 'yl', 'u', 't', 'itty', 'y', 'the', 'i', 'he']
o ethekitt d ee s k kethed doyou gy lik gg og see yl u t itty y the i he
72
27
['doy', 'o', 'us', 'eeth', 'e', 'k', 'i', 'tt', 'y', 'see', 'th', 'edo', 'g', 'gydoyo', 'u', 'li', 'k', 'et', 'he', 'k', 'i', 'tt', 'yliketh', 'ed', 'o', 'gg', 'y']
set(['he', 'yliketh', 'e', 'g', 'i', 'ed', 'k', 'see', 'us', 'edo', 'gg', 'eeth', 'doy', 'u', 'o', 'th', 'y', 'et', 'li', 'tt', 'gydoyo'])
['he', 'yliketh', 'e', 'g', 'i', 'ed', 'k', 'see', 'us', 'edo', 'gg', 'eeth', 'doy', 'u', 'o', 'th', 'y', 'et', 'li', 'tt', 'gydoyo']
he yliketh e g i ed k see us edo gg eeth doy u o th y et li tt gydoyo
69
29
['d', 'o', 'yo', 'use', 'e', 'theki', 't', 'ty', 'seet', 'he', 'dog', 'gy', 'd', 'oy', 'oul', 'i', 'k', 'e', 't', 'h', 'e', 'k', 'itt', 'y', 'li', 'ket', 'h', 'edog', 'gy']
set(['seet', 'oul', 'gy', 'itt', 'use', 'ty', 'theki', 'li', 'yo', 'edog', 'ket', 'he', 'e', 'd',

68
21
['doyou', 'see', 't', 'hekitty', 's', 'ee', 't', 'hedoggy', 'do', 'yo', 'u', 'liketh', 'e', 'k', 'itt', 'y', 'li', 'k', 'e', 'thedog', 'gy']
set(['do', 'e', 'yo', 'hedoggy', 'thedog', 'ee', 'hekitty', 'k', 'liketh', 's', 'gy', 'doyou', 'see', 'u', 't', 'y', 'itt', 'li'])
['do', 'e', 'yo', 'hedoggy', 'thedog', 'ee', 'hekitty', 'k', 'liketh', 's', 'gy', 'doyou', 'see', 'u', 't', 'y', 'itt', 'li']
do e yo hedoggy thedog ee hekitty k liketh s gy doyou see u t y itt li
70
27
['do', 'y', 'o', 'use', 'e', 't', 'hekitt', 'y', 'seet', 'hed', 'oggy', 'doyo', 'u', 'liket', 'he', 'k', 'it', 't', 'y', 'l', 'i', 'ket', 'he', 'd', 'og', 'g', 'y']
set(['seet', 'it', 'liket', 'use', 'ket', 'hed', 'oggy', 'doyo', 'do', 'hekitt', 'he', 'e', 'd', 'g', 'i', 'k', 'l', 'o', 'og', 'u', 't', 'y'])
['seet', 'it', 'liket', 'use', 'ket', 'hed', 'oggy', 'doyo', 'do', 'hekitt', 'he', 'e', 'd', 'g', 'i', 'k', 'l', 'o', 'og', 'u', 't', 'y']
seet it liket use ket hed oggy doyo do hekitt he e d g i k l o og u t y

...

74
25
['do', 'youseethe', 'ki', 'tty', 's', 'eet', 'hed', 'ogg', 'y', 'doyou', 'li', 'ke', 't', 'h', 'ek', 'i', 't', 'ty', 'li', 'ke', 't', 'h', 'e', 'dogg', 'y']
set(['do', 'tty', 'e', 'ek', 'ke', 'ty', 'i', 'eet', 'ki', 'doyou', 'li', 'hed', 's', 't', 'youseethe', 'h', 'y', 'ogg', 'dogg'])
['do', 'tty', 'e', 'ek', 'ke', 'ty', 'i', 'eet', 'ki', 'doyou', 'li', 'hed', 's', 't', 'youseethe', 'h', 'y', 'ogg', 'dogg']
do tty e ek ke ty i eet ki doyou li hed s t youseethe h y ogg dogg
66
25
['d', 'o', 'y', 'ous', 'e', 'et', 'hek', 'i', 't', 't', 'yseethed', 'ogg', 'yd', 'o', 'y', 'o', 'ul', 'ike', 'th', 'ekittyli', 'k', 'e', 'the', 'dog', 'gy']
set(['ul', 'e', 'd', 'yseethed', 'i', 'gy', 'k', 'ekittyli', 'yd', 'o', 'dog', 'ous', 't', 'ike', 'y', 'et', 'ogg', 'th', 'the', 'hek'])
['ul', 'e', 'd', 'yseethed', 'i', 'gy', 'k', 'ekittyli', 'yd', 'o', 'dog', 'ous', 't', 'ike', 'y', 'et', 'ogg', 'th', 'the', 'hek']
ul e d yseethed i gy k ekittyli yd o dog ous t ike y et ogg th the hek
70
23
['doy'

['do', 'e', 'd', 'i', 'h', 'k', 'ggy', 'l', 'kittys', 'ouse', 'u', 'o', 'oggy', 'y', 'eth', 'ket', 'the', 'ittyl']
do e d i h k ggy l kittys ouse u o oggy y eth ket the ittyl
59
27
['d', 'o', 'y', 'ouse', 'et', 'hekit', 't', 'y', 'see', 'th', 'ed', 'o', 'ggydo', 'youli', 'k', 'eth', 'eki', 't', 't', 'yl', 'i', 'k', 'et', 'he', 'd', 'ogg', 'y']
set(['eki', 'd', 'hekit', 'i', 'ed', 'k', 'o', 'ouse', 'ggydo', 'see', 't', 'th', 'youli', 'y', 'et', 'eth', 'he', 'ogg', 'yl'])
['eki', 'd', 'hekit', 'i', 'ed', 'k', 'o', 'ouse', 'ggydo', 'see', 't', 'th', 'youli', 'y', 'et', 'eth', 'he', 'ogg', 'yl']
eki d hekit i ed k o ouse ggydo see t th youli y et eth he ogg yl
65
25
['do', 'y', 'ou', 's', 'eethe', 'k', 'i', 'tt', 'yse', 'ethedo', 'gg', 'y', 'do', 'you', 'lik', 'eth', 'eki', 'tt', 'yl', 'i', 'keth', 'e', 'do', 'gg', 'y']
set(['do', 'eethe', 'ethedo', 'keth', 'yl', 'eki', 'i', 'k', 'yse', 'you', 'gg', 's', 'y', 'eth', 'ou', 'lik', 'tt', 'e'])
['do', 'eethe', 'ethedo', 'keth', 'yl', 'eki', 'i

64
23
['do', 'yo', 'u', 's', 'e', 'et', 'hekit', 'ty', 'seethe', 'doggy', 'do', 'y', 'ouli', 'k', 'e', 'thek', 'it', 't', 'yl', 'i', 'kethe', 'do', 'ggy']
set(['kethe', 'do', 'e', 'hekit', 'yo', 'ty', 'i', 'k', 'ouli', 'ggy', 'it', 'thek', 's', 'u', 't', 'y', 'et', 'seethe', 'doggy', 'yl'])
['kethe', 'do', 'e', 'hekit', 'yo', 'ty', 'i', 'k', 'ouli', 'ggy', 'it', 'thek', 's', 'u', 't', 'y', 'et', 'seethe', 'doggy', 'yl']
kethe do e hekit yo ty i k ouli ggy it thek s u t y et seethe doggy yl
70
31
['do', 'youse', 'eth', 'e', 'ki', 't', 'tyse', 'eth', 'ed', 'og', 'g', 'y', 'd', 'oy', 'oul', 'ike', 't', 'he', 'k', 'i', 't', 't', 'y', 'l', 'i', 'ke', 't', 'hedo', 'g', 'g', 'y']
set(['ike', 'do', 'hedo', 'e', 'd', 'g', 'oul', 'youse', 'ed', 'ki', 'l', 'og', 'ke', 't', 'oy', 'y', 'i', 'eth', 'he', 'k', 'tyse'])
['ike', 'do', 'hedo', 'e', 'd', 'g', 'oul', 'youse', 'ed', 'ki', 'l', 'og', 'ke', 't', 'oy', 'y', 'i', 'eth', 'he', 'k', 'tyse']
ike do hedo e d g oul youse ed ki l og ke t oy y i eth 

['do', 'y', 'ou', 'seethe', 'k', 'ittyse', 'eth', 'e', 'dog', 'g', 'y', 'doyo', 'u', 'l', 'ik', 'e', 't', 'hek', 'i', 'tty', 'l', 'ik', 'e', 't', 'he', 'd', 'o', 'g', 'gy']
set(['tty', 'gy', 'l', 'ik', 'seethe', 'doyo', 'hek', 'do', 'ittyse', 'eth', 'he', 'e', 'd', 'g', 'i', 'k', 'dog', 'o', 'u', 't', 'y', 'ou'])
['tty', 'gy', 'l', 'ik', 'seethe', 'doyo', 'hek', 'do', 'ittyse', 'eth', 'he', 'e', 'd', 'g', 'i', 'k', 'dog', 'o', 'u', 't', 'y', 'ou']
tty gy l ik seethe doyo hek do ittyse eth he e d g i k dog o u t y ou
69
27
['doyo', 'u', 'see', 't', 'h', 'e', 'kit', 'tyseet', 'he', 'do', 'gg', 'y', 'do', 'y', 'o', 'u', 'likethe', 'k', 'it', 't', 'yliket', 'h', 'e', 'd', 'o', 'g', 'gy']
set(['gy', 'do', 'e', 'likethe', 'gg', 'h', 'k', 'it', 'o', 'kit', 'tyseet', 'd', 'see', 'u', 't', 'y', 'yliket', 'doyo', 'g', 'he'])
['gy', 'do', 'e', 'likethe', 'gg', 'h', 'k', 'it', 'o', 'kit', 'tyseet', 'd', 'see', 'u', 't', 'y', 'yliket', 'doyo', 'g', 'he']
gy do e likethe gg h k it o kit tyseet d see

'0000000000000001000000000010000000000000000100000000000'